In [1]:
# Face Recognition

import cv2
import os
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import random
import pygame
from pygame import mixer
from pygame.locals import*

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') # We load the cascade for the face.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32, 5)
        self.conv2 = nn.Conv2d(32,64, 5)
        self.conv3 = nn.Conv2d(64,128, 5)
       
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        #rint(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


PATH  =  'firsttModel'.replace('/','\ ').replace(' ','')
net = torch.load(PATH)
net.eval()

to_delete=[]

def normalize_score(score):
    score = 1/score
    while(score < 10):
        score *= 10
        if score< 10:
            score = 9**score
    
    while(score > 10):
        score /= 1.5
    return score


def check_pic(name):
    #print("checking")
    #print(name)
    Pic_Path  =  (name+'.jpg').replace('/','\ ').replace(' ','')
    IMG_SIZE = 50
    img = cv2.imread(Pic_Path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img_array = np.array(img)
    
    Xx= torch.Tensor([i for i in (img_array)]).view(-1,1,50,50)#.to(device)
    Xx = Xx/255.0
    
    net_out = net(Xx[0].view(-1,1,50,50))[0]
    predicted_class = torch.argmax(net_out)
    #print(predicted_class)
    sc = float(net_out[1].item())
    #sc = normalize_score(sc)
    #print('attractive score out of 10 : ', sc)
    #print(' % : ', print(net_out[1].item()))
    #print('\n\n ' + '='*40 + '\n\n')
    return sc



def get_files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file


    
def run(folder):
    global to_delete
    ranking = {}

    
    TestPath =  folder+'//'
    for file in get_files(TestPath):
        if file.endswith(".jpg"):
            try:
                #ranking[file] = check_pic(file.split('.')[0])
                img = cv2.imread(folder+'//'+file)
                ranking[file] = detect(img)
                remove_files(to_delete)
            except Exception as e:
                #print(e)
                pass

    #print (ranking)

    max_score = max(ranking, key=ranking.get) 

    print (max_score)
    print (ranking[max_score])
    return ranking
    #show(max_score)

    
def show(picture):

    Pic_Path = 'images/'.replace('/','\ ').replace(' ','')
    Pic_Path += picture #+ '.jpg'

    from PIL import Image
    image = Image.open(Pic_Path)
    image.show()


def remove_files(files):
    for file in files:
        os.remove(file)


def detect(img):
    global to_delete
    #print("detecting")
    counter=0
    current_score = 0
    faces = face_cascade.detectMultiScale(img, 1.3, 5) # We apply the detectMultiScale method from the face cascade to locate one or several faces in the image.
    #print(len(faces))
    for (x, y, w, h) in faces: # For each detected face:
        #cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) # We paint a rectangle around the face.
        crop_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp_'+str(counter)+'.jpg', crop_img)
        current_score += check_pic('temp_'+str(counter))
        to_delete.append('temp_'+str(counter)+'.jpg')
        counter+=1
    if len(faces)>0:
        return current_score /len(faces)
    else:
        cv2.imwrite('temp_'+str(counter)+'.jpg', img)
        to_delete.append('temp_'+str(counter)+'.jpg')
        return check_pic(img)




ranks = run("images")
print('ranks: ',ranks)


face6067.jpg
0.4104028344154358
ranks:  {'000038.jpg': 0.02989131025969982, 'att.jpg': 0.05314796417951584, 'att1.jpg': 0.1491527147591114, 'att11.jpg': 0.008831012062728405, 'face6017.jpg': 0.10724888741970062, 'face6018.jpg': 0.09702885150909424, 'face6022.jpg': 0.11383000016212463, 'face6065.jpg': 0.17382338643074036, 'face6066.jpg': 0.2969013750553131, 'face6067.jpg': 0.4104028344154358, 'face6068.jpg': 0.10808272659778595, 'face6070.jpg': 0.028897415846586227, 'face6071.jpg': 0.07461721450090408, 'face6073.jpg': 0.04354490339756012, 'face6075.jpg': 0.05484728887677193, 'face6077.jpg': 0.034526530653238297, 'face6078.jpg': 0.06325134634971619, 'face6080.jpg': 0.0427732840180397, 'face6081.jpg': 0.09321516752243042, 'face6082.jpg': 0.01325688511133194, 'fam.jpg': 0.11546317301690578, 'khalil.jpg': 0.04769223928451538, 'pretty0.jpg': 0.07614020258188248, 'test0.jpg': 0.02218043664470315, 'ugg.jpg': 0.13057949021458626, 'ugl.jpg': 0.05673655867576599}


In [ ]:



original_list = []
names = []
for root, dirs, files in os.walk("images"):
    for filename in files:
        names.append(filename)
        
start_screen_imgs = []
for i in range(28):
    start_screen_imgs.append(pygame.image.load("assets//start//frame_"+str(i)+".png"))
        
#initializing pygame
pygame.init()

backgrounds = ['assets//bck_0.png' ,'assets//bck_1.jpg','assets//bck_2.jpg','assets//bck_3.jpg']
sort_pics=    ["assets//sort_0.png","assets//sort_1.png","assets//sort_2.png","assets//sort_3.png"]
top_pics =    ["assets//top_0.png" ,"assets//top_1.png","assets//top_2.png","assets//top_3.png"]

#Load background image - icon imgs
theme_number = 0
check_sorted = False
count_till_start=0
k=0


def sort_list(mylist, myranks):
    global original_list, check_sorted
    if check_sorted == False:
        original_list = mylist
        
        templist = list(myranks.items())
        sorted_by_second = sorted(templist, key=lambda tup: tup[1])
        final_result=[]
        for i in range(len(sorted_by_second)-1,-1,-1):
            final_result.append(sorted_by_second[i][0])
        check_sorted = True
        #print('sorted list = ', final_result)
        return final_result
    else:
        check_sorted = False
        return original_list

class grid:
    
    #grid size
    num_of_imgs = len(names)
    #margin between squares in pixels
#     margin = 3
    #screen width and height
    screen_w = 500    #1080/5
    screen_h = 660 #1920/5
    box_side = 4*screen_w/15
#     temp_jump = 0
    boxes =[]
    main_boxes=[]

    

    def prepare_grid(self):
        self.main_boxes.append(pygame.Rect(0, 0 , self.screen_w, self.screen_h/10))
        self.main_boxes.append(pygame.Rect(self.screen_w-100, self.screen_h-70 , 100, 50))
        for i in range(self.num_of_imgs):
            x_picture  = self.screen_w/15 + (self.screen_w/30 + self.box_side) * (i%3)
            y_picture  = self.screen_w/15 + (self.screen_w/30 + self.box_side) * math.floor(i/3)
            self.boxes.append(pygame.Rect(x_picture, y_picture+50 , self.box_side, self.box_side))
            self.box_state = "show"        
            
        
            
class game_main:
    running = True
    score_value = 0
    font = pygame.font.Font("assets//Sportive-Regular.ttf", 32)
    font2 = pygame.font.Font("assets//font2.otf", 100)
        
    #font size
    TextX = 10
    TextY = 10
    score_value = 0
    
    #show game over on the screen
    game_o = font2.render("GAME OVER",True, (0, 0, 0))
    
    #creating game window
    screen = pygame.display.set_mode((grid.screen_w,grid.screen_h))
        
    #change title
    pygame.display.set_caption("OmRika")
        
    #change icon
    icon = pygame.image.load("assets//icon.png") #define icon
    pygame.display.set_icon(icon)             #set the icon


    #Load background music
    mixer.music.load("assets//bck_music.mp3")
    mixer.music.play(-1)
    
    def game_over(self, x,y):
        self.screen.fill((255,255,255,0.5)) #change background color - RGB
        self.screen.blit(self.game_o, (x, y))

    def show_score(self, x,y):
        score = font.render("Score : " + str(self.score_value),True, (255, 255, 255))
        self.screen.blit(score, (x, y))
    
def main():

    my_grid = grid()
    my_grid.prepare_grid()
    main_g = game_main()
    
    
main() 
scroll_y = 0
scroll_bk = 0

#start screen
for count_till_start in range(5400):
    try:
        if count_till_start%60 == 0:
            k+=1
        current_frame = start_screen_imgs[k]
        game_main.screen.blit(current_frame, (0, 0))
        pygame.display.update()   #to update screen
    except Exception as e:
        #k=0
        break
        #print(count_till_start)
        #print(e)

while game_main.running:
    #INSIDE OF THE GAME LOOP
    if scroll_bk>=grid.screen_h/2:
        scroll_bk=0
    else:
        scroll_bk+=1
    bg = pygame.image.load(backgrounds[theme_number])
    sort_pic = pygame.image.load(sort_pics[theme_number])
    top_pic  = pygame.image.load(top_pics[theme_number])
    game_main.screen.blit(bg, (0, scroll_bk))
    for event in pygame.event.get():      #get all user events
        try:
            if event.type == pygame.QUIT:     # pressing the close button = Pygame.Quit
                running = False               #quit the window
                pygame.display.quit()
                pygame.quit()       #closing window - add quit event
                break
            #Scroll down / up
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4: scroll_y = min(scroll_y + 50, 0)
                if event.button == 5: scroll_y = max(scroll_y - 50, -300)
            #get MOUSE clicks
            elif event.type == pygame.MOUSEBUTTONUP:
                pos = pygame.mouse.get_pos()
                check=False
                #collidepoint(x, y)
                for box in grid.main_boxes:
                    if box.collidepoint(pos):
                        #print("You did CLICK this main box ",box)
                        if box == grid.main_boxes[0]:
                            if theme_number+1 < len(backgrounds):
                                #Load different background image - icon imgs
                                theme_number+=1
                            else:
                                theme_number = 0
                        elif box == grid.main_boxes[1]:
                            names = sort_list(names,ranks)
                            #print(names)
                                
                        check=True
                if check ==False:
                     for box in grid.boxes:
                        if box.collidepoint(pos):
                            #print("You did CLICK the box number: ",grid.boxes.index(box))
                            check=True

        except Exception as e:
            print(e)

    #anything that you want to stay inside thescreen need to be inside the while loop

    try:  
        temp_boxes=[]
        for box in grid.boxes:
            try:
                #pygame.draw.rect(game_main.screen, [0, 255, 0], box, 0)
                j=grid.boxes.index(box)
                img = pygame.image.load("images//"+names[j])
                img = pygame.transform.scale(img, (round(grid.box_side), round(grid.box_side)))
                x_pic  = grid.screen_w/15 + (grid.screen_w/30 + grid.box_side) * (j%3)
                y_pic  = grid.screen_w/15 + (grid.screen_w/30 + grid.box_side) * math.floor(j/3) 
                temp_boxes.append(pygame.Rect(x_pic, y_pic+50+scroll_y , grid.box_side, grid.box_side))
                game_main.screen.blit(img,(x_pic, y_pic+50+scroll_y))      
            except:
                pass
        grid.boxes = temp_boxes  
        for boxx in grid.main_boxes:
            pygame.draw.rect(game_main.screen, [222, 22, 229], boxx, 0)
        game_main.screen.blit(sort_pic,(grid.screen_w-100, grid.screen_h-70))
        game_main.screen.blit(top_pic,(0, 0))   
        #pygame.draw.rect(screen, [0, 255, 0], boxes[0], 0)
        pygame.display.update()   #to update screen
    except Exception as e:
        print(e)



sorted list =  ['face6067.jpg', 'face6066.jpg', 'face6065.jpg', 'att1.jpg', 'ugg.jpg', 'fam.jpg', 'face6022.jpg', 'face6068.jpg', 'face6017.jpg', 'face6018.jpg', 'face6081.jpg', 'pretty0.jpg', 'face6071.jpg', 'face6078.jpg', 'ugl.jpg', 'face6075.jpg', 'att.jpg', 'khalil.jpg', 'face6073.jpg', 'face6080.jpg', 'face6077.jpg', '000038.jpg', 'face6070.jpg', 'test0.jpg', 'face6082.jpg', 'att11.jpg']
['face6067.jpg', 'face6066.jpg', 'face6065.jpg', 'att1.jpg', 'ugg.jpg', 'fam.jpg', 'face6022.jpg', 'face6068.jpg', 'face6017.jpg', 'face6018.jpg', 'face6081.jpg', 'pretty0.jpg', 'face6071.jpg', 'face6078.jpg', 'ugl.jpg', 'face6075.jpg', 'att.jpg', 'khalil.jpg', 'face6073.jpg', 'face6080.jpg', 'face6077.jpg', '000038.jpg', 'face6070.jpg', 'test0.jpg', 'face6082.jpg', 'att11.jpg']
['000038.jpg', 'att.jpg', 'att1.jpg', 'att11.jpg', 'face6017.jpg', 'face6018.jpg', 'face6022.jpg', 'face6023.jpg', 'face6065.jpg', 'face6066.jpg', 'face6067.jpg', 'face6068.jpg', 'face6070.jpg', 'face6071.jpg', 'face6072.